# Clean data

This notebook cleans the raw data downloaded from [ISED Canada](https://ised-isde.canada.ca/site/trade-data-online/en) and the [HS2](https://en.wikipedia.org/wiki/Harmonized_System) product categorization codes.

In [1]:
from lxml import html
import os
import pandas as pd

In [2]:
data_dir = os.path.abspath('../data')
misc_dir = os.path.abspath('../misc')

In [17]:
src = os.path.join(misc_dir, 'hs.html')

with open(src, 'r') as f:
    html_text = f.read()
    
doc = html.fromstring(html_text)
texts = doc.xpath('option/text()')

hs = {}
for text in texts:

    if text[0].isnumeric():
        hs[top].append(text)
    else:
        top = text
        hs[text] = []

# clean out empties
hs = {k : v for k , v in hs.items() if v}

# resulting data
res = []
for sec, codes in hs.items():

    for code in codes:
        
        res.append({'sec' : sec, 'code' : code})

codes = pd.DataFrame(res)

# shorten sectors
short_secs = {
    'I - LIVE ANIMALS AND ANIMAL PRODUCTS': 'I - LIVE ANIMALS AND ANIMAL PRODUCTS',
    'II - VEGETABLE PRODUCTS': 'II - VEGETABLE PRODUCTS',
    'III - FATS, OILS, THEIR CLEAVAGE PRODUCTS AND WAXES': 'III - FATS, OILS AND WAX PRODUCTS',
    'IV - FOOD PRODUCTS, BEVERAGES, SPIRITS, VINEGAR AND TOBACCO PRODUCTS': 'IV - FOOD, BEVERAGES, SPIRITS AND TOBACCO PRODUCTS',
    'V - MINERAL PRODUCTS': 'V - MINERAL PRODUCTS',
    'VI - PRODUCTS OF THE CHEMICAL OR ALLIED INDUSTRIES': 'VI - CHEMICAL PRODUCTS',
    'VII - PLASTICS, RUBBER AND ARTICLES MADE FROM THESE MATERIALS': 'VII - PLASTICS AND RUBBER',
    'VIII - RAW HIDES, SKINS, LEATHER, FUR AND ARTICLES MADE FROM THESE MATERIALS': 'VIII - RAW HIDES, SKINS, LEATHER AND FUR PRODUCTS',
    'IX - WOOD AND WOOD ARTICLES, CORK, STRAW AND OTHER PLAITING MATERIALS': 'IX - LUMBER AND WOOD PRODUCTS',
    'X - WOOD PULP, PAPER AND PAPER ARTICLES': 'X - WOOD PULP AND PAPER PRODUCTS',
    'XI - TEXTILES AND TEXTILE ARTICLES': 'XI - TEXTILES AND TEXTILE PRODUCTS',
    'XII - FOOTWEAR, HEADWEAR, UMBRELLAS, CANES AND SIMILAR ACCESSORIES': 'XII - WEARABLE ACCESSORIES',
    'XIII - GLASS, GLASSWARE;  ARTICLES OF CERAMICS, STONE AND SIMILAR MATERIALS': 'XIII - GLASS, CERAMICS AND STONE WARES',
    'XIV - PEARLS, PRECIOUS METALS OR STONES, COINS AND JEWELLERY': 'XIV - PRECIOUS METALS OR STONES, COINS AND JEWELLERY',
    'XV - BASE METALS AND ARTICLES OF BASE METAL': 'XV - BASE METALS AND ARTICLES OF BASE METAL',
    'XVI - MACHINERY;  MECHANICAL,  ELECTRICAL AND ELECTRONIC APPLIANCES OR EQUIPMENT': 'XVI - MACHINERY AND ELECTRONICS',
    'XVII - VEHICLES, AIRCRAFT, VESSELS AND OTHER TRANSPORTATION EQUIPMENT': 'XVII - VEHICLES AND TRANSPORTATION EQUIPMENT',
    'XVIII - CLOCKS, WATCHES AND SPECIALIZED INSTRUMENTATION': 'XVIII - CLOCKS, WATCHES AND INSTRUMENTATION',
    'XIX - ARMS AND AMMUNITION': 'XIX - ARMS AND AMMUNITION',
    'XX - MISCELLANEOUS MANUFACTURED ARTICLES': 'XX - MISCELLANEOUS MANUFACTURED PRODUCTS',
    "XXI - WORKS OF ART, COLLECTOR'S PIECES AND ANTIQUES": "XXI - ART AND COLLECTOR ITEMS"
}

codes['sec'] = codes['sec'].replace(short_secs)

dst = os.path.join(data_dir, 'HS2_codes.csv')
codes.to_csv(dst, index=False)
print('Saved:', dst)

Saved: /home/qcx201/Projects/CAtrade/data/HS2_codes.csv


In [4]:
raw_dir = os.path.abspath('../raw')
files = sorted(os.listdir(raw_dir))

# breakdown by province vs trade partner
modes = {
    'part' : {'naArea': '9999', 'countryList': 'DET'},
    'prov' : {'naArea': '9998', 'countryList': 'ALL'},
}

# years
years = [str(x) for x in range(2000, 2025)]

# product codes (2-digit zero-padded)
codes = [str(x).zfill(2) for x in range(1, 100)]
codes = [x for x in codes if x not in ('77', '98', '99')] # remove non-valid codes

dfs = {}

for mode in modes:
    for year in years:
        for code in codes:
            
            file = f'{mode}_{year}_{code}.csv'

            src = os.path.join(raw_dir, file)
            print(src)
            
            tmp = pd.read_csv(src, header=6)
            
            # fill missing countries
            tmp.iloc[:, 0:2] = tmp.iloc[:, 0:2].ffill()
            
            # set and rename index
            tmp = tmp.set_index(['Unnamed: 0', 'Unnamed: 1'])
            tmp.index.names = ['place', 'account']
            
            # rename columns
            tmp.columns = pd.MultiIndex.from_tuples([(year, code)])
            
            # remove rows with empty data
            tmp = tmp.dropna()
            
            if mode not in dfs:
                dfs[mode] = tmp.copy()
                print('> added')
            else:
                dfs[mode] = pd.merge(dfs[mode], tmp, how='outer', left_index=True, right_index=True)
                print('> merged')

/home/qcx201/Projects/CAtrade/raw/part_2000_01.csv
> added
/home/qcx201/Projects/CAtrade/raw/part_2000_02.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_03.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_04.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_05.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_06.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_07.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_08.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_09.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_10.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_11.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_12.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_13.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_14.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_15.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_2000_16.csv
> merged
/home/qcx201/Projects/CAtrade/raw/part_20

Some strange duplications occur in the source data, e.g. for Albania, year 2022, code 84. [[source data link]](https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=INDIVIDUAL&searchType=BL&areaCodes=&naArea=9999&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2022&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7C84)

See entries in file: /home/qcx201/Projects/CAtrade/raw/part_2022_84.csv

```csv
"Albania","Total Exports","2077773"
"","Total Imports","957064"
"","Trade Balance","1120709"
"","Total Exports","1137478"
"","Total Imports","22765"
"","Trade Balance","1114713"
```

If you search specifically for Albania code 84, you get the first set of entries. [[source data link]](https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=INDIVIDUAL&searchType=BL&areaCodes=242&naArea=9999&countryList=specific&toFromCountry=CDN&reportType=TB&customYears=2022&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7C84)

So I just keep first entries for now and drop duplicates.

In [5]:
for key in dfs:

    df = dfs[key]

    # keep first set of trade balance entries if duplicated
    # see markdown notes above
    isdup = df.index.duplicated(keep='first')
    df = df[~isdup]

    df = df.T

    # stack data (wide format to long)
    cols = [(acct.split('.')[0], cou) for cou, acct in df]
    df.columns = pd.MultiIndex.from_tuples(cols)
    df = df[sorted(df.columns)]

    df = df.stack(level=1, future_stack=True)
    df = df.dropna(how='all', axis=0)

    # rename index and columns
    df.index.names = ('year', 'hs2', 'cou')
    cols = df.columns
    cols = cols.str.lower()
    cols = cols.str.replace(' ', '')
    cols = cols.str.replace('total', '')
    df.columns = cols

    dst = os.path.join(data_dir, f'CA-{key}.csv')
    df.to_csv(dst)
    print('Saved:', dst)


Saved: /home/qcx201/Projects/CAtrade/data/CA-part.csv
Saved: /home/qcx201/Projects/CAtrade/data/CA-prov.csv
